In [15]:
import tkinter as tk
from PIL import Image, ImageDraw
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from scipy.ndimage import label  # Importing the label function from ndimage

# Load the trained model
model_cnn = load_model('my_cnn_model.keras')

# Create the main window
class DrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Digit Recognition")

        self.canvas = tk.Canvas(root, width=280, height=280, bg='white')
        self.canvas.pack()

        self.button_predict = tk.Button(root, text="Predict", command=self.predict)
        self.button_predict.pack()

        self.image = Image.new("L", (280, 280), 255)
        self.draw = ImageDraw.Draw(self.image)

        self.canvas.bind("<B1-Motion>", self.paint)

        self.predictions = []

    def paint(self, event):
        x, y = event.x, event.y
        self.draw.ellipse([x-5, y-5, x+5, y+5], fill=0)
        self.canvas.create_oval(x-5, y-5, x+5, y+5, fill='black')

    def predict(self):
        # Process the image to detect digits and predict each one
        img_array = np.array(self.image)
        img_array = np.invert(img_array)  # Invert colors if necessary

        # Use thresholding to segment the digits
        threshold = 50
        img_array[img_array > threshold] = 255
        img_array[img_array <= threshold] = 0

        # Find connected components (this assumes simple thresholding can separate digits)
        labeled_array, num_features = label(img_array)  # Using the correct import

        # Loop through each connected component (assumed to be a digit)
        for i in range(1, num_features + 1):
            digit_region = (labeled_array == i).astype(np.uint8) * 255

            # Find bounding box of the digit
            coords = np.column_stack(np.where(digit_region > 0))
            y0, x0 = coords.min(axis=0)
            y1, x1 = coords.max(axis=0)

            digit_image = digit_region[y0:y1+1, x0:x1+1]

            # Resize to 28x28
            img_resized = Image.fromarray(digit_image).resize((28, 28), Image.LANCZOS)
            img_resized = np.array(img_resized) / 255.0
            img_resized = np.expand_dims(img_resized, axis=0)
            img_resized = np.expand_dims(img_resized, axis=-1)  # Add channel dimension

            # Predict the digit
            predictions = model_cnn.predict(img_resized)
            predicted_class = np.argmax(predictions)

            # Store the prediction and its position
            self.predictions.append((predicted_class, (x0, y1)))

        # Update canvas to show predictions
        self.display_predictions()

    def display_predictions(self):
        for pred, (x, y) in self.predictions:
            self.canvas.create_text(x, y + 10, text=str(pred), fill="red", font=("Helvetica", 16))

# Run the app
root = tk.Tk()
app = DrawingApp(root)
root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
